Inainte de executarea laboratorului, rulati:
 - pip install scipy
 - pip install audio_metadata
 
Dezarhivati continutul arhivei *samples.zip* de pe Moodle intr-un folder **samples** aflat la aceeasi locatie ca notebook-ul.

## Functii procesare

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wave
from scipy.io import wavfile
import contextlib
import os
import IPython


# from http://stackoverflow.com/questions/2226853/interpreting-wav-data/2227174#2227174
def interpret_wav(raw_bytes, n_frames, n_channels, sample_width, interleaved = True):

    if sample_width == 1:
        dtype = np.uint8 # unsigned char
    elif sample_width == 2:
        dtype = np.int16 # signed 2-byte short
    else:
        raise ValueError("Only supports 8 and 16 bit audio formats.")

    channels = np.frombuffer(raw_bytes, dtype=dtype)

    if interleaved:
        # channels are interleaved, i.e. sample N of channel M follows sample N of channel M-1 in raw data
        channels.shape = (n_frames, n_channels)
        channels = channels.T
    else:
        # channels are not interleaved. All samples from channel M occur before all samples from channel M-1
        channels.shape = (n_channels, n_frames)

    return channels

def get_start_end_frames(nFrames, sampleRate, tStart=None, tEnd=None):

    if tStart and tStart*sampleRate<nFrames:
        start = tStart*sampleRate
    else:
        start = 0

    if tEnd and tEnd*sampleRate<nFrames and tEnd*sampleRate>start:
        end = tEnd*sampleRate
    else:
        end = nFrames

    return (start,end,end-start)

def extract_audio(fname, tStart=None, tEnd=None):
    with contextlib.closing(wave.open(fname,'rb')) as spf:
        sampleRate = spf.getframerate()
        ampWidth = spf.getsampwidth()
        nChannels = spf.getnchannels()
        nFrames = spf.getnframes()

        startFrame, endFrame, segFrames = get_start_end_frames(nFrames, sampleRate, tStart, tEnd)

        # Extract Raw Audio from multi-channel Wav File
        spf.setpos(startFrame)
        sig = spf.readframes(segFrames)
        spf.close()

        channels = interpret_wav(sig, segFrames, nChannels, ampWidth, True)

        return (channels, nChannels, sampleRate, ampWidth, nFrames)

def convert_to_mono(channels, nChannels, outputType):
    if nChannels == 2:
        print('Fisierul audio are 2 canale')
        samples = np.mean(np.array([channels[0], channels[1]]), axis=0)  # Convert to mono
        dif = np.sum(np.abs(np.array(channels[0]) - np.array(channels[1])))
        if( dif == 0 ):
            print('\tCele doua canale sunt identice')
        else:
            print('\tExista diferente intre cele 2 canale')
    else:
        print('Fisierul audio are 1 canal')
        samples = channels[0]

    return samples.astype(outputType)

def plot_specgram(samples, sampleRate, tStart=None, tEnd=None):
    plt.figure(figsize=(20,10))
    plt.specgram(samples, Fs=sampleRate, NFFT=1024, noverlap=192, cmap='nipy_spectral', xextent=(tStart,tEnd))
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

def plot_audio_samples(title, samples, sampleRate, tStart=None, tEnd=None):
    if not tStart:
        tStart = 0

    if not tEnd or tStart>tEnd:
        tEnd = len(samples)/sampleRate

    f, axarr = plt.subplots(2, sharex=True, figsize=(20,10))
    axarr[0].set_title(title)
    axarr[0].plot(np.linspace(tStart, tEnd, len(samples)), samples)
    axarr[1].specgram(samples, Fs=sampleRate, NFFT=1024, noverlap=192, cmap='nipy_spectral', xextent=(tStart,tEnd))
    #get_specgram(axarr[1], samples, sampleRate, tStart, tEnd)

    axarr[0].set_ylabel('Amplitude')
    axarr[1].set_ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')

    plt.show()
    
def plot_two_channels(title, channel1, channel2, sampleRate, tStart=None, tEnd=None):
    if not tStart:
        tStart = 0

    if not tEnd or tStart>tEnd:
        tEnd = len(samples)/sampleRate
        
    plt.figure(figsize=(14, 5), dpi=80)
    plt.title(title)
    plt.plot(np.linspace(tStart, tEnd, len(channel1)), channel1, 'b')
    plt.plot(np.linspace(tStart, tEnd, len(channel2)), channel2, 'r')

    plt.ylabel('Amplitude')
    plt.xlabel('Time [sec]')

    plt.show()\
    
def get_audio_duration( file_name ):
    with contextlib.closing(wave.open(file_name,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        return (duration, rate)
    return (0, 0)


## Audio 1 - semnal dreptunghiular

#### [Play audio] Square wave - original

In [ ]:
audio_file = 'samples/square_wave.wav'

IPython.display.Audio( audio_file )

#### Afisare informatii si grafice amplitudine si frecventa

In [ ]:
duration, sample_rate = get_audio_duration( audio_file )
bit_depth = 16
nb_channels = 2
bitrate = sample_rate * bit_depth * nb_channels
estimated_dim = int( ( duration * sample_rate * bit_depth * nb_channels ) / 8 )
file_dim = os.path.getsize(audio_file)

# Interval de secvente de timp de analizat
tStart = 0
tEnd = 1

print( "Durata audio: ", duration, "s, Frecventa: ", sample_rate, "Hz, Bitrate: ", bitrate, \
      "bps, Dimensiune calculata: ", estimated_dim, "B" )
print("Dimensiune fisier pe disk: ", file_dim, 'B')
print("Metadate: ", file_dim - estimated_dim, 'b')

channels, nChannels, sampleRate, ampWidth, nFrames = extract_audio(audio_file, tStart, tEnd)
one_channel = convert_to_mono(channels, nChannels, np.int16)

plot_audio_samples("Grafice square wave", one_channel, sampleRate, tStart, tEnd)

wavfile.write('samples/square_wave_mono.wav', sampleRate, one_channel)

#### Detaliere si afisare canale

In [ ]:
channel1 = channels[0].astype(np.int16)
channel2 = channels[1].astype(np.int16)

plot_two_channels("Square wave - 2 canale suprapuse", channel1, channel2, sampleRate, tStart, tEnd)

wavfile.write('samples/square_wave_channel1.wav', sampleRate, channel1.T)
wavfile.write('samples/square_wave_channel2.wav', sampleRate, channel2.T)

#### [Play audio] Square Wave - mono channel

In [ ]:
IPython.display.Audio("samples/square_wave_mono.wav")

#### [Play audio] Square Wave - canal 1

In [ ]:
IPython.display.Audio("samples/square_wave_channel1.wav")

#### [Play audio] Square Wave - canal 2

In [ ]:
IPython.display.Audio("samples/square_wave_channel2.wav")

#### Metadate fisier square wave

In [ ]:
import audio_metadata

metadata = audio_metadata.load( audio_file )
print(metadata)
print(metadata.streaminfo.bit_depth)

## Audio 2

In [ ]:
audio_file = 'samples/Queen_Bohemian_Rhapsody.wav'
# TODO: Play audio

In [ ]:
# TODO: determinare si afisare informatii: durata, bitrate, sample rate, dimensiune\

# Interval de secvente de timp de analizat
tStart = 188
tEnd = 205

# TODO: taiere fisier in intervalul de timp [tStart, tEnd] si extragere canale

# TODO : convetire la un singur canal

# TODO: plotare amplitudine si frecventa in interval [tStart, tEnd]

# Salvare audio canal mono
mono_channel_filename = 'samples/Queen_Bohemian_mono.wav'


#### Verificare fisier salvat

In [ ]:
IPython.display.Audio(mono_channel_filename)

In [ ]:
channels_m, nChannels_m, sampleRate_m, ampWidth_m, nFrames_m = extract_audio(mono_channel_filename, tStart, tEnd)
one_channel_m = convert_to_mono(channels_m, nChannels_m, np.int16)

In [ ]:
metadata = audio_metadata.load( audio_file )
print(metadata)

#### Verificare si afisare canale

In [ ]:
# TODO: extragere canale

# TODO: Plotare canale suprapuse

# TODO: Scriere fisere separate pentru fiecare canal

#### [Play Audio] Bohemian Rapsody - canal 1

#### [Play Audio] Bohemian Rapsody - canal 2

## Fisier MP3

In [ ]:
audio_file = 'samples/Queen_Bohemian_Rhapsody.mp3'
metadata_mp3 = audio_metadata.load( audio_file )
print("Metadate MP3: ", metadata_mp3)

audio_file = 'samples/Queen_Bohemian_Rhapsody.wav'
metadata_wav = audio_metadata.load( audio_file )
print("Metadate WAV: ", metadata_wav)

print("Diferente samplerate: \t", metadata_mp3.streaminfo.sample_rate, " \tvs.\t ", metadata_wav.streaminfo.sample_rate)
print("Diferente bitrate: \t", metadata_mp3.streaminfo.bitrate, " \tvs.\t ", metadata_wav.streaminfo.bitrate)
print("Diferente dimesniune: \t", metadata_mp3.filesize, " \tvs.\t ", metadata_wav.filesize)